In [16]:
import pandas as pd
import duckdb as db
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [17]:
# Connect to the clinisys_all database
path_to_db = '../../database/clinisys_all.duckdb'

print(f"Connected to database: {path_to_db}")
print(f"Database file exists: {os.path.exists(path_to_db)}")

# Use context manager for automatic connection cleanup
with db.connect(path_to_db, read_only=True) as conn:
    # List all schemas
    schemas = conn.execute("SELECT schema_name FROM information_schema.schemata").fetchdf()
    print("\n=== Available Schemas ===")
    print(schemas)

    # List all tables in silver schema
    silver_tables = conn.execute("""
        SELECT table_name, table_type 
        FROM information_schema.tables 
        WHERE table_schema = 'silver'
        ORDER BY table_name
    """).fetchdf()

    print(f"\n=== Tables in Silver Schema ({len(silver_tables)} tables) ===")
    print(silver_tables)

    # Explore each table in silver schema
    print("\n" + "="*80)
    print("EXPLORING SILVER SCHEMA TABLES")
    print("="*80)

    for idx, row in silver_tables.iterrows():
        table_name = row['table_name']
        table_type = row['table_type']
        
        print(f"\n{'='*60}")
        print(f"TABLE: {table_name} (Type: {table_type})")
        print(f"{'='*60}")
        
        try:
            # Get table schema
            schema_info = conn.execute(f"""
                SELECT column_name, data_type, is_nullable
                FROM information_schema.columns 
                WHERE table_schema = 'silver' AND table_name = '{table_name}'
                ORDER BY ordinal_position
            """).fetchdf()
            
            print(f"\nSchema ({len(schema_info)} columns):")
            # print(schema_info.to_string(index=False))
            
            # Get row count
            count_result = conn.execute(f"SELECT COUNT(*) as row_count FROM silver.{table_name}").fetchdf()
            row_count = count_result['row_count'].iloc[0]
            print(f"\nTotal rows: {row_count:,}")
            
            # Get sample data (first 10 rows)
            if row_count > 0:
                sample_data = conn.execute(f"SELECT * FROM silver.{table_name} LIMIT 10").fetchdf()
                print(f"\nSample data (first 10 rows):")
                print(sample_data.to_string(index=False))
                
                # Show data types and null counts
                print(f"\nData info:")
                print(f"Shape: {sample_data.shape}")
                print(f"Memory usage: {sample_data.memory_usage(deep=True).sum() / 1024:.2f} KB")
                
                # Show null counts for each column
                null_counts = sample_data.isnull().sum()
                if null_counts.sum() > 0:
                    print(f"\nNull counts in sample:")
                    for col, null_count in null_counts.items():
                        if null_count > 0:
                            print(f"  {col}: {null_count}")
            else:
                print("\nTable is empty - no sample data to show")
                
        except Exception as e:
            print(f"Error exploring table {table_name}: {str(e)}")
        
        print(f"\n{'-'*60}")

print(f"\n{'='*80}")
print("EXPLORATION COMPLETE")
print(f"{'='*80}")

Connected to database: ../../database/clinisys_all.duckdb
Database file exists: True

=== Available Schemas ===
          schema_name
0              bronze
1                gold
2                main
3              silver
4  information_schema
5                main
6          pg_catalog
7                main

=== Tables in Silver Schema (21 tables) ===
                           table_name  table_type
0         view_congelamentos_embrioes  BASE TABLE
1           view_congelamentos_ovulos  BASE TABLE
2            view_congelamentos_semen  BASE TABLE
3     view_congelamentos_semen_doador  BASE TABLE
4      view_descongelamentos_embrioes  BASE TABLE
5        view_descongelamentos_ovulos  BASE TABLE
6            view_embrioes_congelados  BASE TABLE
7                         view_exames  BASE TABLE
8   view_extrato_atendimentos_central  BASE TABLE
9                   view_medicamentos  BASE TABLE
10      view_medicamentos_prescricoes  BASE TABLE
11                       view_medicos  BASE T

In [18]:
#!/usr/bin/env python3
"""
Extract Prontuario Data - Modular Functions
Extract data for a specific prontuario from finops tables with individual functions for each table.
"""

import pandas as pd
import duckdb as db
import os

def get_database_connection():
    """
    Create and return a connection to the clinisys_all database
    
    Returns:
        duckdb.DuckDBPyConnection: Database connection
    """
    path_to_db = '../../database/clinisys_all.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def get_database_context():
    """
    Create a context manager for the clinisys_all database connection
    
    Returns:
        context manager: Database connection context manager
    """
    path_to_db = '../../database/clinisys_all.duckdb'
    return db.connect(path_to_db, read_only=True)

def extract_orcamentos_data(conn, target_prontuario):
    """
    Extract orcamentos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing orcamentos data
    """
    print(f"\n2. EXTRACTING VIEW_ORCAMENTOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_orc = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_orcamentos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_orcamentos columns: {list(schema_orc['column_name'])}")
        
        # Extract data
        orcamentos_df = conn.execute(f"""
            SELECT * FROM silver.view_orcamentos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(orcamentos_df)} records in view_orcamentos")
        if len(orcamentos_df) > 0:
            print("Sample data:")
            print(orcamentos_df.head())
            print(f"Columns: {list(orcamentos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting orcamentos data: {str(e)}")
        orcamentos_df = pd.DataFrame()
    
    return orcamentos_df

def extract_extrato_atendimento_central_data(conn, target_prontuario):
    """
    Extract extrato_atendimento_central data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing extrato_atendimento_central data
    """
    print(f"\n3. EXTRACTING VIEW_EXTRATO_ATENDIMENTO_CENTRAL DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_eac = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_extrato_atendimentos_central'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_extrato_atendimento_central columns: {list(schema_eac['column_name'])}")
        
        # Extract data
        extrato_atendimento_central_df = conn.execute(f"""
            SELECT * FROM silver.view_extrato_atendimentos_central 
            WHERE prontuario = {target_prontuario}
            ORDER BY data DESC, inicio DESC
        """).fetchdf()
        
        print(f"Found {len(extrato_atendimento_central_df)} records in view_extrato_atendimento_central")
        if len(extrato_atendimento_central_df) > 0:
            print("Sample data:")
            print(extrato_atendimento_central_df.head())
            print(f"Columns: {list(extrato_atendimento_central_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting extrato_atendimento_central data: {str(e)}")
        extrato_atendimento_central_df = pd.DataFrame()
    
    return extrato_atendimento_central_df

def extract_tratamentos_data(conn, target_prontuario):
    """
    Extract tratamentos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing tratamentos data
    """
    print(f"\n4. EXTRACTING VIEW_TRATAMENTOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_trat = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_tratamentos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_tratamentos columns: {list(schema_trat['column_name'])}")
        
        # Extract data
        tratamentos_df = conn.execute(f"""
            SELECT * FROM silver.view_tratamentos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(tratamentos_df)} records in view_tratamentos")
        if len(tratamentos_df) > 0:
            print("Sample data:")
            print(tratamentos_df.head())
            print(f"Columns: {list(tratamentos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting tratamentos data: {str(e)}")
        tratamentos_df = pd.DataFrame()
    
    return tratamentos_df

def extract_congelamentos_embrioes_data(conn, target_prontuario):
    """
    Extract congelamentos_embrioes data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing congelamentos_embrioes data
    """
    print(f"\n5. EXTRACTING VIEW_CONGELAMENTOS_EMBRIOES DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_ce = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_congelamentos_embrioes'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_congelamentos_embrioes columns: {list(schema_ce['column_name'])}")
        
        # Extract data
        congelamentos_embrioes_df = conn.execute(f"""
            SELECT * FROM silver.view_congelamentos_embrioes 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(congelamentos_embrioes_df)} records in view_congelamentos_embrioes")
        if len(congelamentos_embrioes_df) > 0:
            print("Sample data:")
            print(congelamentos_embrioes_df.head())
            print(f"Columns: {list(congelamentos_embrioes_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting congelamentos_embrioes data: {str(e)}")
        congelamentos_embrioes_df = pd.DataFrame()
    
    return congelamentos_embrioes_df

def extract_congelamentos_ovulos_data(conn, target_prontuario):
    """
    Extract congelamentos_ovulos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing congelamentos_ovulos data
    """
    print(f"\n6. EXTRACTING VIEW_CONGELAMENTOS_OVULOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_co = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_congelamentos_ovulos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_congelamentos_ovulos columns: {list(schema_co['column_name'])}")
        
        # Extract data
        congelamentos_ovulos_df = conn.execute(f"""
            SELECT * FROM silver.view_congelamentos_ovulos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(congelamentos_ovulos_df)} records in view_congelamentos_ovulos")
        if len(congelamentos_ovulos_df) > 0:
            print("Sample data:")
            print(congelamentos_ovulos_df.head())
            print(f"Columns: {list(congelamentos_ovulos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting congelamentos_ovulos data: {str(e)}")
        congelamentos_ovulos_df = pd.DataFrame()
    
    return congelamentos_ovulos_df

def extract_descongelamentos_embrioes_data(conn, target_prontuario):
    """
    Extract descongelamentos_embrioes data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing descongelamentos_embrioes data
    """
    print(f"\n7. EXTRACTING VIEW_DESCONGELAMENTOS_EMBRIOES DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_de = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_descongelamentos_embrioes'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_descongelamentos_embrioes columns: {list(schema_de['column_name'])}")
        
        # Extract data
        descongelamentos_embrioes_df = conn.execute(f"""
            SELECT * FROM silver.view_descongelamentos_embrioes 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(descongelamentos_embrioes_df)} records in view_descongelamentos_embrioes")
        if len(descongelamentos_embrioes_df) > 0:
            print("Sample data:")
            print(descongelamentos_embrioes_df.head())
            print(f"Columns: {list(descongelamentos_embrioes_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting descongelamentos_embrioes data: {str(e)}")
        descongelamentos_embrioes_df = pd.DataFrame()
    
    return descongelamentos_embrioes_df

def extract_descongelamentos_ovulos_data(conn, target_prontuario):
    """
    Extract descongelamentos_ovulos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing descongelamentos_ovulos data
    """
    print(f"\n8. EXTRACTING VIEW_DESCONGELAMENTOS_OVULOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_do = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_descongelamentos_ovulos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_descongelamentos_ovulos columns: {list(schema_do['column_name'])}")
        
        # Extract data
        descongelamentos_ovulos_df = conn.execute(f"""
            SELECT * FROM silver.view_descongelamentos_ovulos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(descongelamentos_ovulos_df)} records in view_descongelamentos_ovulos")
        if len(descongelamentos_ovulos_df) > 0:
            print("Sample data:")
            print(descongelamentos_ovulos_df.head())
            print(f"Columns: {list(descongelamentos_ovulos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting descongelamentos_ovulos data: {str(e)}")
        descongelamentos_ovulos_df = pd.DataFrame()
    
    return descongelamentos_ovulos_df

def extract_embrioes_congelados_data(conn, target_prontuario):
    """
    Extract embrioes_congelados data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing embrioes_congelados data
    """
    print(f"\n9. EXTRACTING VIEW_EMBRIOES_CONGELADOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_ec = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_embrioes_congelados'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_embrioes_congelados columns: {list(schema_ec['column_name'])}")
        
        # Extract data
        embrioes_congelados_df = conn.execute(f"""
            SELECT * FROM silver.view_embrioes_congelados 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(embrioes_congelados_df)} records in view_embrioes_congelados")
        if len(embrioes_congelados_df) > 0:
            print("Sample data:")
            print(embrioes_congelados_df.head())
            print(f"Columns: {list(embrioes_congelados_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting embrioes_congelados data: {str(e)}")
        embrioes_congelados_df = pd.DataFrame()
    
    return embrioes_congelados_df

def extract_prontuario_data(target_prontuario=876950):
    """
    Extract all data for a specific prontuario from the finops tables:
    - view_orcamentos  
    - view_extrato_atendimento_central
    - view_tratamentos
    - view_congelamentos_embrioes
    - view_congelamentos_ovulos
    - view_descongelamentos_embrioes
    - view_descongelamentos_ovulos
    - view_embrioes_congelados
    
    Args:
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        tuple: Eight DataFrames containing the extracted data
    """
    
    print(f"\n{'='*80}")
    print(f"EXTRACTING DATA FOR PRONTUARIO: {target_prontuario}")
    print(f"{'='*80}")
    
    # Use context manager for automatic connection cleanup
    with get_database_context() as conn:
        # Extract data from each table using individual functions
        orcamentos_df = extract_orcamentos_data(conn, target_prontuario)
        extrato_atendimento_central_df = extract_extrato_atendimento_central_data(conn, target_prontuario)
        tratamentos_df = extract_tratamentos_data(conn, target_prontuario)
        congelamentos_embrioes_df = extract_congelamentos_embrioes_data(conn, target_prontuario)
        congelamentos_ovulos_df = extract_congelamentos_ovulos_data(conn, target_prontuario)
        descongelamentos_embrioes_df = extract_descongelamentos_embrioes_data(conn, target_prontuario)
        descongelamentos_ovulos_df = extract_descongelamentos_ovulos_data(conn, target_prontuario)
        embrioes_congelados_df = extract_embrioes_congelados_data(conn, target_prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"SUMMARY FOR PRONTUARIO {target_prontuario}")
        print(f"{'='*80}")
        print(f"view_orcamentos: {len(orcamentos_df)} records")
        print(f"view_extrato_atendimento_central: {len(extrato_atendimento_central_df)} records")
        print(f"view_tratamentos: {len(tratamentos_df)} records")
        print(f"view_congelamentos_embrioes: {len(congelamentos_embrioes_df)} records")
        print(f"view_congelamentos_ovulos: {len(congelamentos_ovulos_df)} records")
        print(f"view_descongelamentos_embrioes: {len(descongelamentos_embrioes_df)} records")
        print(f"view_descongelamentos_ovulos: {len(descongelamentos_ovulos_df)} records")
        print(f"view_embrioes_congelados: {len(embrioes_congelados_df)} records")
        
        print(f"\n{'='*80}")
        print("DATA EXTRACTION COMPLETE")
        print(f"{'='*80}")
        print("DataFrames available:")
        print("- orcamentos_df") 
        print("- extrato_atendimento_central_df")
        print("- tratamentos_df")
        print("- congelamentos_embrioes_df")
        print("- congelamentos_ovulos_df")
        print("- descongelamentos_embrioes_df")
        print("- descongelamentos_ovulos_df")
        print("- embrioes_congelados_df")
        
        return (orcamentos_df, extrato_atendimento_central_df, tratamentos_df, 
                congelamentos_embrioes_df, congelamentos_ovulos_df, 
                descongelamentos_embrioes_df, descongelamentos_ovulos_df, 
                embrioes_congelados_df)

def find_in_diario_by_all_ids(prontuario: int) -> pd.DataFrame:
    """
    Find all records in diario_vendas by all ids in the patient_id list.
    
    Args:
        patient_id (int): The patient's id
        
    Returns:
        pd.DataFrame: DataFrame containing all diario_vendas records for the patient
    """
    HUNTINGTON_DB_PATH = '../../database/huntington_data_lake.duckdb'
    
    try:
        # Then, get all records from diario_vendas for these codigos
        with db.connect(HUNTINGTON_DB_PATH, read_only=True) as con:
            # Create the IN clause for the query
           
            query_diario = f"""
            SELECT *
            FROM silver.mesclada_vendas
            WHERE "prontuario" = {prontuario}
            ORDER BY "DT Emissao" DESC
            """
            
            df_diario = con.execute(query_diario).df()

            # df_diario['flag_same_prontuario'] = df_diario['Cliente']==prontuario
            
            if df_diario.empty:
                print(f"No records found in silver.mesclada_vendas for prontuario {prontuario}")
                return df_diario
            
            print(f"Found {len(df_diario)} records in silver.mesclada_vendas for prontuario {prontuario}")
            return df_diario
            

    except Exception as e:
        print(f"Error retrieving diario_vendas data for prontuario {prontuario}: {e}")
        return pd.DataFrame()

# if __name__ == "__main__":
#     # Example usage
#     (orcamentos_df, extrato_atendimento_central_df, tratamentos_df, 
#      congelamentos_embrioes_df, congelamentos_ovulos_df, 
#      descongelamentos_embrioes_df, descongelamentos_ovulos_df, 
#      embrioes_congelados_df) = extract_prontuario_data(876950)
    
#     # You can now work with the DataFrames
#     print("\n DataFrames loaded successfully!")
#     print(f"orcamentos_df shape: {orcamentos_df.shape}")
#     print(f"extrato_atendimento_central_df shape: {extrato_atendimento_central_df.shape}")
#     print(f"tratamentos_df shape: {tratamentos_df.shape}")
#     print(f"congelamentos_embrioes_df shape: {congelamentos_embrioes_df.shape}")
#     print(f"congelamentos_ovulos_df shape: {congelamentos_ovulos_df.shape}")
#     print(f"descongelamentos_embrioes_df shape: {descongelamentos_embrioes_df.shape}")
#     print(f"descongelamentos_ovulos_df shape: {descongelamentos_ovulos_df.shape}")
#     print(f"embrioes_congelados_df shape: {embrioes_congelados_df.shape}")
    
#     # Example of using the diario function
#     diario_df = find_in_diario_by_all_ids(876950)
#     print(f"diario_df shape: {diario_df.shape}") 

In [19]:

patient_id = 142631  

(orcamentos_df, extrato_atendimento_central_df, tratamentos_df,
congelamentos_embrioes_df, congelamentos_ovulos_df,
descongelamentos_embrioes_df, descongelamentos_ovulos_df,
embrioes_congelados_df) = extract_prontuario_data(patient_id)

diario_df = find_in_diario_by_all_ids(patient_id)


EXTRACTING DATA FOR PRONTUARIO: 142631

2. EXTRACTING VIEW_ORCAMENTOS DATA
--------------------------------------------------
view_orcamentos columns: ['id', 'prontuario', 'paciente', 'clinica', 'tipo_cotacao', 'profissional', 'status', 'status_entrega', 'nome_contato', 'telefone_contato', 'email_contato', 'comentario_para_paciente', 'comentario_do_paciente', 'orcamento_texto', 'descricao', 'fornecedor', 'qtd_cotada', 'unidade', 'valor_unidade', 'total', 'data_entrega', 'centro_custos', 'valor_total', 'forma', 'parcelas', 'comentarios', 'forma_parcela', 'valor', 'data_pagamento', 'descricao_pagamento', 'data', 'responsavel', 'data_entrega_orcamento', 'data_ultima_modificacao', 'hash', 'extraction_timestamp']
Found 3 records in view_orcamentos
Sample data:
      id  prontuario paciente     clinica tipo_cotacao profissional  status  \
0  39084      142631   esposa  0000000001         None          481  E-mail   
1  32784      142631   esposa  0000000001         None            8  E-mail

In [20]:
orcamentos_df

,id,prontuario,paciente,clinica,tipo_cotacao,profissional,status,status_entrega,nome_contato,telefone_contato,email_contato,comentario_para_paciente,comentario_do_paciente,orcamento_texto,descricao,fornecedor,qtd_cotada,unidade,valor_unidade,total,data_entrega,centro_custos,valor_total,forma,parcelas,comentarios,forma_parcela,valor,data_pagamento,descricao_pagamento,data,responsavel,data_entrega_orcamento,data_ultima_modificacao,hash,extraction_timestamp
0,39084,142631,esposa,0000000001,None,481,E-mail,Realizado,None,None,None,"<p>22/07/2024 - Pagou R$&nbsp;18.646,45 no din...",None,<p><img alt=&quot;&quot; src=&quot;data:image/...,0000076890|@|,|@|,|@|,<NA>,"18.646,45|@|","18.646,45|@|",NaT,1,"18.646,45",Dinheiro,None,None,None,NaN,NaT,None,2024-07-22,3737,2024-07-22,2024-12-06,322e9a6fe649bbfd7b85a30bc5ba8c6d,2025-07-21 21:40:47
1,32784,142631,esposa,0000000001,None,8,E-mail,Realizado,None,None,None,<p><strong>13/01/2024</strong> - Mediante a co...,None,<p><span style=&quot;font-size:14px&quot;><img...,0000075997|@|,|@|,|@|,<NA>,"25.947,00|@|","25.947,00|@|",NaT,1,"25.947,00",Dinheiro,None,None,None,NaN,NaT,None,2024-01-13,5615,2024-01-13,2024-12-05,aa7e3edb59fec9d9f4257e8679152cd4,2025-07-21 21:40:47
2,23091,142631,esposa,0000000001,None,481,None,Realizado,None,None,None,"<p><strong>31/01/2023</strong>- Pagou R$920,00...",None,None,0000075995|@|,|@|,|@|,<NA>,"11.505,00|@|","11.505,00|@|",NaT,1,"11.505,00",Dinheiro,None,None,None,NaN,NaT,None,2023-03-31,4126,NaT,2024-01-13,5307faebee991ce3f077dd513a0e44d1,2025-07-21 21:40:47


In [21]:
tratamentos_df 

,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,hora_procedimento,tipo_procedimento,motivo_congelamento,tipo_inducao,protocolo,tipo_ciclo_congelado,tipo_preparo_endometrial,num_congelados,num_transferir,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,material_heterologo,utero_substituicao,prontuario_genitores,prontuario_uter_substituicao,doacao_ovulos,destino_doacao_ovulos,prontuario_receptora,complemento_receptora,doacao_embrioes,era,emma,alice,matrice_lab,imunohistoquimica,outros_biopsia,consentimento,financeiro,embryoscope,pgt,era_auxiliar,recomendacao_transferencia,resultado_era,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,fator_infertilidade3,fsh_exame,foliculos_antrais,data_histeroscopia,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,ropa,data_inicio_agonista,gnrh,fsh_agonista,lh_agonista,outros_medicamentos,gonal,menogon,menopur,pergoveris,elonva,serofene_clomid_indux,luveris,puregon,letrozole,fostimon,merional,rekovelle,descricao_outras_medicacoes,dose_total_fsh,dose_total_lh,dose_total_fsh_lh,coorte_e2,coorte_aco,coorte_progesterona,coq10,hormonal_combinado,coorte_testosterona,agonista_coorte,coorte_antagonista,coorte_dhea,utilizado_hcg,motivo_cancelamento_hcg,data_hcg,horario_planejado_hcg,horario_real_hcg,ovidrel,gonapeptyl,dose_gonapeptyl,outras_complicacoes_hcg,dose_outras_complicacoes,lupron,outros_hcg,descricao_outros_hcg,data_inicio_progesterona,p4_inicio_progesterona,estradiol_fase_lutea,progesterona_fase_lutea,progesterona_oral,progesterona_vaginal,progesterona_injetavel,combinacao_oral_vaginal,nenhuma_progesterona,hcg_progesterona,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,observacoes_descongelamento,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,p4_transferencia,motivo_nao_transferir,motivo_cancelamento_tratamento,ligacao_enfermagem,beta_valor,beta_valor2,data_ultrassom1,data_ultrassom2,num_sacos_gestacionais,tsh,data_ultrassom1_contato,descricao_ultrassom1,bcf_embrião1_status,bcf_embrião1_fonte,data_ultrassom2_contato,descricao_ultrassom2,data_ultrassom3_contato,descricao_ultrassom3,resultado_tratamento,analise_genetica,resultado_analise_genetica,motivo_perda_aborto,data_nascimento_bebes,obstetra,nome_bebe1,sexo_bebe1,peso_bebe1,altura_bebe1,idade_gestacional_bebe1,via_parto_bebe1,viabilidade_bebe1,anomalias_bebe1,nome_bebe2,sexo_bebe2,peso_bebe2,altura_bebe2,idade_gestacional_bebe2,via_parto_bebe2,viabilidade_bebe2,anomalias_bebe2,nome_bebe3,sexo_bebe3,peso_bebe3,altura_bebe3,idade_gestacional_bebe3,via_parto_bebe3,viabilidade_bebe3,anomalias_bebe3,nome_bebe4,sexo_bebe4,peso_bebe4,altura_bebe4,idade_gestacional_bebe4,via_parto_bebe4,viabilidade_bebe4,anomalias_bebe4,observacoes_bebes,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,30769,142631,1,38,37,esposa,4,2024-12-05,None,Ciclo de Congelados,None,None,None,None,Hormonal,None,None,None,None,NaN,None,None,None,Homólogo,None,Não,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,PGT-A,Não,None,None,Não,Não,None,None,None,None,"0,00","0,00",None,2024-11-12,None,None,None,Outros,None,None,None,None,None,2024-11-14,22,esposa,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6,05/12/2024,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None

In [22]:
extrato_atendimento_central_df[extrato_atendimento_central_df['confirmado'] == 1]

,agendamento_id,data,inicio,data_agendamento_original,medico,medico2,prontuario,evento,evento2,centro_custos,agenda,chegou,confirmado,paciente_codigo,paciente_nome,medico_nome,medico_sobrenome,medico2_nome,centro_custos_nome,agenda_nome,procedimento_nome,hash,extraction_timestamp
1,1305028,2024-12-16,1900-01-01 06:00:00,NaT,4034,<NA>,142631.0,765557,None,1,43,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Enfermagem,Betinha,None,1. HTT SP - Ibirapuera,Agenda Exames - IBIRAPUERA,Acompanhamento Beta HCG,d44cbdecc278ada249a8a9be074eb54c,2025-07-21 21:39:24
3,1305704,2024-12-10,1900-01-01 07:20:00,NaT,4034,<NA>,142631.0,765557,None,1,43,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Enfermagem,Betinha,None,1. HTT SP - Ibirapuera,Agenda Exames - IBIRAPUERA,Acompanhamento Beta HCG,bda445198a1c1f28c06ff1adbeacb18a,2025-07-21 21:39:24
4,1303904,2024-12-05,1900-01-01 14:00:00,NaT,3533,<NA>,142631.0,765568,None,1,75,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Claudia,Gomes,None,1. HTT SP - Ibirapuera,Claudia Gomes - IBIRAPUERA,FET: Ciclo Próprio,f84169aeae654de9efda2729986fce1a,2025-07-21 21:39:24
5,1303903,2024-12-05,1900-01-01 14:00:00,NaT,<NA>,8,142631.0,765568,None,1,72,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,None,None,Claudia Gomes Padilla,1. HTT SP - Ibirapuera,Sala de Procedimento 01 - IBIRAPUERA,FET: Ciclo Próprio,af9a0e4b7549b2af0b3f4f7a8aa1970c,2025-07-21 21:39:24
8,1300272,2024-12-04,1900-01-01 06:00:00,NaT,4034,<NA>,142631.0,765557,None,1,43,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Enfermagem,Betinha,None,1. HTT SP - Ibirapuera,Agenda Exames - IBIRAPUERA,Acompanhamento Beta HCG,bc11dcdd8c4de7622092fc4af912a775,2025-07-21 21:39:24
9,1298652,2024-11-28,1900-01-01 08:30:00,NaT,3533,<NA>,142631.0,765450,None,1,432,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Claudia,Gomes,None,1. HTT SP - Ibirapuera,Bebê à Distância,6º US Ciclo,ec340943fd74c9c2c85d86e5cf4f35a2,2025-07-21 21:39:24
10,1296845,2024-11-25,1900-01-01 08:30:00,NaT,3533,<NA>,142631.0,765449,None,1,432,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Claudia,Gomes,None,1. HTT SP - Ibirapuera,Bebê à Distância,5º US Ciclo,0a05953425066e10d9d771a49b20eaf8,2025-07-21 21:39:24
11,1295073,2024-11-23,1900-01-01 07:30:00,NaT,3533,<NA>,142631.0,765448,None,1,432,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Claudia,Gomes,None,1. HTT SP - Ibirapuera,Bebê à Distância,4º US Ciclo,2bcae85533751770f2a77961a5a970dd,2025-07-21 21:39:24
12,1294070,2024-11-21,1900-01-01 08:30:00,NaT,3533,<NA>,142631.0,765447,None,1,432,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Claudia,Gomes,None,1. HTT SP - Ibirapuera,Bebê à Distância,3º US Ciclo,0a0e2308eec01c5ca6cc323959692373,2025-07-21 21:39:24
13,1290764,2024-11-19,1900-01-01 07:00:00,NaT,3533,<NA>,142631.0,765446,None,1,432,Atendido,1,142631,Sausan Hassan Hussein Jaber Thalji,Claudia,Gomes,None,1. HTT SP - Ibirapuera,Bebê à Distância,2º US Ciclo,c4c56a863a3d839a92e07f515f5d044a,2025-07-21 21:39:24


In [23]:
congelamentos_embrioes_df

,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Ovulo,D2,D3,D4,D5,D6,D7,rack,rack2,rack3,rack4,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,responsavel_congelamento_d5,responsavel_checagem_d5,responsavel_congelamento_d6,responsavel_checagem_d6,responsavel_congelamento_d7,responsavel_checagem_d7,hash,extraction_timestamp
0,26129,E1893/24,1,142631,esposa,2024-07-31,1900-01-01 14:25:00,2406/24,Não,None,None,None,<NA>,NaT,<NA>,NaT,5,5,E21336,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,NaN,NaN,NaN,None,<NA>,Izadora Reis,Beatriz Aiello,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,a7dcc3255f66072290a0cf7e886f007d,2025-07-21 21:38:47
1,20805,E14593,1,142631,esposa,2019-12-13,1900-01-01 14:00:00,None,Não,None,None,None,<NA>,NaT,<NA>,NaT,3,3,E14593,NU,4,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,NaN,NaN,NaN,None,<NA>,Marina Fanelli,Embriologista TI,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7de7ec0083c4bfcb5aaf0aafd8949257,2025-07-21 21:38:47


In [24]:
descongelamentos_embrioes_df

,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Transferencia,DataTransferencia,Prateleira,Incubadora,transferidos_transferencia,cateter_transferencia,lote_transferencia,validade_transferencia,intercorrencia_transferencia,sangue_interno_transferencia,sangue_externo_transferencia,retorno_transferencia,vezes_retorno_transferencia,Transfer_D5,responsavel_transferencia,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,16546,2675/24,1,142631,0,2024-07-31,2024-12-05,2406/24,E21336,E1893/24,YU,6,1,DESVITRIFICAÇÃO,<NA>,2024-12-05,<NA>,12.6,1,SYDNEY,None,None,None,Não,Não,Não,None,None,3719.0,None,Otavio Santos,Bruna Arroio,2dd9297efa9b1b5ecfd022aa732429fe,2025-07-21 21:38:51
1,11176,IB546/23,1,142631,0,2019-12-13,2023-03-31,-,E14593,E14593,NU,4,1,DESVITRIFICAÇÃO,<NA>,2023-03-31,<NA>,12.1,1,SYDNEY,None,None,None,Não,Não,Não,None,None,3719.0,None,Anaíde Silva Sousa,Thiago Placido,16e696d833322c03938e14d7e4973a5d,2025-07-21 21:38:51


In [25]:
embrioes_congelados_df

,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,75221,208511,26129,0,142631,P19,E21336,<NA>,#19,None,None,None,Blastocisto Grau 3 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"Não portador - Aneuplóide complexo, XX - NÃO a...",Sim,Aneuploide,4.8,D6,1.48,YU,6,E21336,PGT-M + PGT-A,<NA>,ecf550c4fae32cca7f12880412b3e903,2025-07-21 21:38:57
1,75220,208510,26129,0,142631,P18,E21336,<NA>,#18,None,None,None,Blastocisto Grau 4 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"Não portador - Anormal/Aneuplóide, XX - NÃO ap...",Sim,Aneuploide,4.0,D6,1.98,YU,6,E21336,PGT-M + PGT-A,<NA>,451896b094ba6d757d6c0543b943d3a2,2025-07-21 21:38:57
2,75219,208504,26129,0,142631,P12,E21336,<NA>,#12,None,None,None,Blastocisto Grau 4 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"Portador - Anormal/Aneuplóide, XY - NÃO apto p...",Sim,Aneuploide,4.9,D6,1.6,YU,6,E21336,PGT-M + PGT-A,<NA>,5c8843e8223023f05f12285032919499,2025-07-21 21:38:57
3,75218,208503,26129,0,142631,P11,E21336,<NA>,#11,None,None,None,Blastocisto Grau 4 - B - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"Não portador - Anormal/Aneuplóide, XY- NÃO apt...",Sim,Aneuploide,3.3,D6,7.82,YU,6,E21336,PGT-M + PGT-A,<NA>,3542d812eea656b11ef6b79ef1cd9ad1,2025-07-21 21:38:57
4,75217,208501,26129,16546,142631,P9,E21336,<NA>,#9,None,None,None,Blastocisto Grau 4 - C - B,<NA>,None,"EUPLOIDE, Masculino",Sim,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Blastocisto Grau 4,C,B,None,None,None,None,<NA>,None,None,None,None,None,Transferido,Sim,Euploide,"Não portador - Normal/Euplóide, XY - Apto para ET",Sim,Euploide,3.6,D6,2.37,YU,6,E21336,PGT-M + PGT-A,<NA>,783a8a8901246a59beb16cfee416ddb8,2025-07-21 21:38:57
5,56718,0,20805,11176,142631,P4,E14593,<NA>,#4,None,None,None,B4CC,<NA>,None,"EUPLOIDE, Feminino",Sim,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Blastocisto Grau 4,C,C,None,None,None,None,<NA>,None,None,None,None,None,Transferido,Sim,Normal,"Portador, Euploide, Feminino",Sim,Normal,None,D6,None,None,None,None,None,<NA>,d360a8483ed6829e3f8ba5de64cd0680,2025-07-21 21:38:57
6,56717,0,20805,0,142631,P3,E14593,<NA>,#3,None,None,None,B4CB,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"Normal, Mosaico de baixo grau: -5, X",Sim,Mosaico,None,D6,None,None,None,None,None,<NA>,25973145100d852e5e2de8dc1b0203e9,2025-07-21 21:38:57
7,56716,0,20805,0,142631,P2,E14593,<NA>,#2,None,None,None,B4BC,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA

In [26]:
congelamentos_ovulos_df

,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp


In [27]:
descongelamentos_ovulos_df 


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp


In [28]:
diario_df

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-12-06,CRINONE GEL VAGINAL 15 APLICADORES (C.E.,1.0,750.00,Estimulação,10101,510,230553,RPS,229147,7,CLAUDIA GOMES PADILLA,042250,,000000000004607,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,328689,20251022_110944,Mescladas2 (3).xlsx
1,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-12-05,APLICACAO DE TRACTOCILE (ATOSIBANO),1.0,1061.00,Estimulação,10101,502,230537,RPS,229129,7,CLAUDIA GOMES PADILLA,042250,,S3560.13,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100030,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,328669,20251022_110944,Mescladas2 (3).xlsx
2,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-12-03,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,10101,502,229796,RPS,228390,0,CRIOPRESERVACAO,042250,,S3566.39,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,327848,20251022_110944,Mescladas2 (3).xlsx
3,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-11-04,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,10101,502,225387,RPS,223987,0,CRIOPRESERVACAO,042250,,S3566.39,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,322915,20251022_110944,Mescladas2 (3).xlsx
4,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-10-03,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,10101,502,220741,RPS,219358,0,CRIOPRESERVACAO,042250,,S3566.39,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,317726,20251022_110944,Mescladas2 (3).xlsx
5,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-09-03,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,10101,502,215839,RPS,214479,0,CRIOPRESERVACAO,042250,,S3566.39,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Anuidade - Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,312338,20251022_110944,Mescladas2 (3).xlsx
6,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-09-02,CRIOPRESERVACAO MEDICOS INTERNOS,1.0,2470.00,Criopreservação,10101,502,215782,RPS,214424,0,CRIOPRESERVACAO,042250,,S3566.04,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100020,,,Criopreservação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,312272,20251022_110944,Mescladas2 (3).xlsx
7,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-08-25,BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,5.0,9400.00,Biópsia Embrionária,10101,502,214652,RPS,213292,7,CLAUDIA GOMES PADILLA,042250,,S3563.09,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100024,,,Biópsia - Embriões,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,310942,20251022_110944,Mescladas2 (3).xlsx
8,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-07-26,"ORGALUTRAN 0,25MG 0,5ML 1 SER PRENC C 0,",2.0,780.00,Estimulação,10101,510,210750,RPS,209402,7,CLAUDIA GOMES PADILLA,042250,,000000000004624,0.0,0.0,0.0,0.00,Ibirapuera,0,0,11310100005,,,Estimulação,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,306516,20251022_110944,Mescladas2 (3).xlsx
9,44355,OMAR HASSAN SA,44281,SAUSAN HASSAN HUSSEIN JABER THALJI,142631,2024-07-26,ZYMOT,1.0,808.00,Zymot,10101,502,210751,RPS,209403,7,CLAUDIA GOMES PADILLA,042250,,S3573.04,0.0,0.0,0.0,0.00,Ibirapuera,0,0,31110100017,,,Zymot,0,0,1,PRESTACAO DE SERVICO,None,NaT,False,306517

In [29]:
diario_df.groupby('Descricao').agg(
    {'Cta-Ctbl': 'count', 'Total': 'sum'}).sort_values(by='Total', ascending=False)

,Cta-Ctbl,Total
Descricao,,
FIV - MEDICOS INTERNOS - 2º CICLO,1,25947.00
COLETA ADICIONAL - MEDICOS INTERNOS - 3,1,18646.45
FET - EMBRIAO EXCEDENTE,1,11505.00
BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,1,9400.00
EMBRYOSCOPE,2,5025.00
"ORGALUTRAN 0,25MG 0,5ML 1 SER PRENC C 0,",5,2945.00
CRIOPRESERVACAO MEDICOS INTERNOS,1,2470.00
"GONAL F PEN 300UI 22MCG 0,5ML (C.E.)",2,2220.00
MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,21,2200.00


In [30]:
# Connect to huntington_data_lake for embryo timeline
data_lake_path = '../../database/huntington_data_lake.duckdb'

# Query embryo freeze timeline for a specific prontuario
prontuario = 182925  # Change this to any prontuario you want to analyze

with db.connect(data_lake_path, read_only=True) as conn_data_lake:
    embryo_timeline_df = conn_data_lake.execute(f"""
        SELECT *
        FROM gold.embryo_freeze_timeline
        WHERE prontuario = {prontuario}
        ORDER BY period_month DESC
    """).fetchdf()

print(f"=== EMBRYO FREEZE TIMELINE FOR PRONTUARIO {prontuario} ===")
print(f"Total records: {len(embryo_timeline_df)}")
print(embryo_timeline_df.to_string(index=False))


=== EMBRYO FREEZE TIMELINE FOR PRONTUARIO 182925 ===
Total records: 7
 prontuario period_month  embryos_frozen_count  embryos_unfrozen_count  embryos_discarded_count  embryos_storage_balance
     182925      2025-01                   0.0                       1                        0                      0.0
     182925      2024-08                   0.0                       1                        0                      1.0
     182925      2022-08                   0.0                       1                        0                      2.0
     182925      2022-01                   0.0                       1                        0                      3.0
     182925      2021-12                   4.0                       0                        0                      4.0
     182925      2021-10                   1.0                       1                        0                      0.0
     182925      2021-07                   5.0                       0             